In [1]:
library(dplyr)
library(readr)
library(tictoc)
library(stringr)
library(tidyr)


Attaching package: ‘dplyr’

The following objects are masked from ‘package:stats’:

    filter, lag

The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union



In [2]:
mutate_where <- function(.data, condition, ..., envir = parent.frame()) {
  condition <- eval(substitute(condition), .data, envir)
  .data[condition, ] <- .data[condition, ] %>% mutate(...)
  .data
}

In [3]:
matches.raw <- read_csv('results.csv')

Parsed with column specification:
cols(
  date = col_date(format = ""),
  home_team = col_character(),
  away_team = col_character(),
  home_score = col_integer(),
  away_score = col_integer(),
  tournament = col_character(),
  city = col_character(),
  country = col_character(),
  neutral = col_logical()
)


In [4]:
length(union(matches.raw$home_team,matches.raw$away_team))

[1] 244

In [5]:
foot.teams <- data.frame(team=union(matches.raw$home_team,matches.raw$away_team),elo=1000,match=0)

In [7]:
newelo <- function(eloA,eloB,scoreA,K=32) {
    EA <- 1/(1+10**((eloB-eloA)/400))
    delta <- round((scoreA-EA)*K,0)
    c(eloA+delta,eloB-delta)
}

In [8]:
dim(matches.raw)

[1] 39054     9

In [9]:
matches.selected <- matches.raw

In [10]:
foot.teams <- data.frame(team=union(matches.selected$home_team,matches.selected$away_team),elo=1000,match=0,w=0,l=0,d=0)
tic('elo rating')
for (row in 1:(dim(matches.selected)[1])) {
    current <- matches.selected[row,]
    scoreA <- (sign(current$home_score-current$away_score)+1)/2
    eloA <- foot.teams[foot.teams$team==current$home_team,]$elo
    eloB <- foot.teams[foot.teams$team==current$away_team,]$elo
    nelo <- newelo(eloA,eloB,scoreA)
    #cat(paste(current$home_team,'-',current$away_team,':',eloA,eloB,'->',nelo[1],nelo[2],'\n'))
    foot.teams[foot.teams$team==current$home_team,]$elo <- nelo[1]
    foot.teams[foot.teams$team==current$away_team,]$elo <- nelo[2]
    foot.teams[foot.teams$team==current$home_team,]$match <- foot.teams[foot.teams$team==current$home_team,]$match +1    
    foot.teams[foot.teams$team==current$away_team,]$match <- foot.teams[foot.teams$team==current$away_team,]$match +1
    if (scoreA==1) {
        foot.teams[foot.teams$team==current$home_team,]$w = foot.teams[foot.teams$team==current$home_team,]$w +1
        foot.teams[foot.teams$team==current$away_team,]$l = foot.teams[foot.teams$team==current$away_team,]$l +1
    } else if (scoreA==0) {
        foot.teams[foot.teams$team==current$home_team,]$l = foot.teams[foot.teams$team==current$home_team,]$l +1
        foot.teams[foot.teams$team==current$away_team,]$w = foot.teams[foot.teams$team==current$away_team,]$w +1
    } else {
        foot.teams[foot.teams$team==current$home_team,]$d = foot.teams[foot.teams$team==current$home_team,]$d +1
        foot.teams[foot.teams$team==current$away_team,]$d = foot.teams[foot.teams$team==current$away_team,]$d +1
    }
}
toc()

elo rating: 159.975 sec elapsed


In [12]:
foot.teams$team <- as.character(foot.teams$team)

In [42]:
foot.teams %>% mutate(wpct=round(100*w/match,2),lpct=round(100*l/match,2),dpct=round(100*d/match,2))%>% arrange(desc(elo))

team,elo,match,w,l,d,wpct,lpct,dpct
Brazil,1554,952,607,154,191,63.76,16.18,20.06
Spain,1479,671,390,127,154,58.12,18.93,22.95
France,1450,813,398,241,174,48.95,29.64,21.40
Germany,1445,930,545,194,191,58.60,20.86,20.54
Argentina,1432,953,512,204,237,53.73,21.41,24.87
England,1430,977,554,185,238,56.70,18.94,24.36
Belgium,1417,745,313,269,163,42.01,36.11,21.88
Portugal,1399,594,285,171,138,47.98,28.79,23.23
Peru,1379,598,194,256,148,32.44,42.81,24.75
Colombia,1367,528,201,181,146,38.07,34.28,27.65


### Loop for Stats

In [13]:
newelo <- function(eloA,eloB,scoreA,K=32) {
    EA <- 1/(1+10**((eloB-eloA)/400))
    delta <- round((scoreA-EA)*K,0)
    c(eloA+delta,eloB-delta)
}

In [14]:
probadraw <- function(eloA,eloB) {
    pwin <- 1/(1+10**((eloB-eloA)/400))
    delta <- abs(2*pwin-1)
    (1-delta)*exp(-delta)/2    
}

In [15]:
create_pool_round <- function() {
    countries <- c('Russia','Saudi Arabia','Egypt','Uruguay','Portugal','Spain','Morocco','Iran',
                   'France','Australia','Peru','Denmark','Argentina','Iceland','Croatia','Nigeria',
                   'Brazil','Switzerland','Costa Rica','Serbia','Germany','Mexico','Sweden','Korea Republic',
                   'Belgium','Panama','Tunisia','England','Poland','Senegal','Colombia','Japan' )
    poolteam <- c('A','A','A','A','B','B','B','B',
                  'C','C','C','C','D','D','D','D',
                  'E','E','E','E','F','F','F','F',
                  'G','G','G','G','H','H','H','H')
    pools <- data.frame(team=countries,pool=poolteam,stringsAsFactors=F)
    
    wc.teams <<- foot.teams %>% inner_join(pools,by='team') %>% mutate(match=0,w=0,l=0,d=0)

    date <- c('Jun 14 2018','Jun 15 2018','Jun 15 2018','Jun 15 2018','Jun 16 2018','Jun 16 2018','Jun 16 2018','Jun 16 2018',
              'Jun 17 2018','Jun 17 2018','Jun 17 2018','Jun 18 2018','Jun 18 2018','Jun 18 2018','Jun 19 2018','Jun 19 2018',
              'Jun 19 2018','Jun 20 2018','Jun 20 2018','Jun 20 2018','Jun 21 2018','Jun 21 2018','Jun 21 2018','Jun 22 2018',
              'Jun 22 2018','Jun 22 2018','Jun 23 2018','Jun 23 2018','Jun 23 2018','Jun 24 2018','Jun 24 2018','Jun 24 2018',
              'Jun 25 2018','Jun 25 2018','Jun 25 2018','Jun 25 2018','Jun 26 2018','Jun 26 2018','Jun 26 2018','Jun 26 2018',
              'Jun 27 2018','Jun 27 2018','Jun 27 2018','Jun 27 2018','Jun 28 2018','Jun 28 2018','Jun 28 2018','Jun 28 2018')
    home_team <- c('Russia','Egypt','Portugal','Morocco','France','Peru','Argentina','Croatia',
                   'Brazil','Costa Rica','Germany','Sweden','Belgium','Tunisia','Poland','Colombia',
                   'Russia','Uruguay','Portugal','Iran','France','Denmark','Argentina','Nigeria',
                   'Brazil','Serbia','Germany','Korea Republic','Belgium','England','Poland','Japan',
                   'Uruguay','Saudi Arabia','Iran','Spain','Denmark','Australia','Nigeria','Iceland',
                   'Serbia','Switzerland','Korea Republic','Mexico','England','Panama','Japan','Senegal')
    away_team <- c('Saudi Arabia','Uruguay','Spain','Iran','Australia','Denmark','Iceland','Nigeria',
                   'Switzerland','Serbia','Mexico','Korea Republic','Panama','England','Senegal','Japan',
                   'Egypt','Saudi Arabia','Morocco','Spain','Peru','Australia','Croatia','Iceland',
                   'Costa Rica','Switzerland','Sweden','Mexico','Tunisia','Panama','Colombia','Senegal',
                   'Russia','Egypt','Portugal','Morocco','France','Peru','Argentina','Croatia',
                   'Brazil','Costa Rica','Germany','Sweden','Belgium','Tunisia','Poland','Colombia')
    pool <- c('A','A','B','B','C','C','D','D','E','E','F','F','G','G','H','H',
              'A','A','B','B','C','C','D','D','E','E','F','F','G','G','H','H',
              'A','A','B','B','C','C','D','D','E','E','F','F','G','G','H','H')

    wc.pool <<- data.frame(pool=pool, date=date, home_team=home_team, away_team=away_team, stringsAsFactors=F) %>% mutate(date=as.Date(date,'%b %d %Y'))
}

In [16]:
play_pool_round <- function() {
    for (row in 1:(dim(wc.pool)[1])) {
        home_team_idx = as.numeric(rownames(wc.teams[wc.teams$team==wc.pool[row,]$home_team,]))
        away_team_idx = as.numeric(rownames(wc.teams[wc.teams$team==wc.pool[row,]$away_team,]))

        wc.pool[row,'home_elo'] <- wc.teams[home_team_idx,]$elo
        wc.pool[row,'away_elo'] <- wc.teams[away_team_idx,]$elo
        wc.pool[row,'home_p'] <- 1/(1+10**((wc.pool[row,]$away_elo-wc.pool[row,]$home_elo)/400))
        wc.pool[row,'draw_p'] <- probadraw(wc.pool[row,]$home_elo,wc.pool[row,]$away_elo)
        wc.pool[row,'home_pwin']  <- wc.pool[row,]$home_p - wc.pool[row,]$draw_p/2
        wc.pool[row,'home_pdraw'] <- wc.pool[row,]$home_p + wc.pool[row,]$draw_p/2
        wc.pool[row,'rand'] <- runif(1)

        wc.pool[row,'home_pts'] <- if_else(wc.pool[row,]$rand<=wc.pool[row,]$home_pwin,1,if_else(wc.pool[row,]$rand<=wc.pool[row,]$home_pdraw,0.5,0))

        nelo <- newelo(wc.pool[row,]$home_elo,wc.pool[row,]$away_elo,wc.pool[row,]$home_pts)
        wc.teams[home_team_idx,]$elo <- nelo[1]
        wc.teams[away_team_idx,]$elo <- nelo[2]

        wc.teams[home_team_idx,]$match <- wc.teams[home_team_idx,]$match +1    
        wc.teams[away_team_idx,]$match <- wc.teams[away_team_idx,]$match +1
        if (wc.pool[row,]$home_pts==1) {
            wc.teams[home_team_idx,]$w = wc.teams[home_team_idx,]$w +1
            wc.teams[away_team_idx,]$l = wc.teams[away_team_idx,]$l +1
        } else if (wc.pool[row,]$home_pts==0) {
            wc.teams[home_team_idx,]$l = wc.teams[home_team_idx,]$l +1
            wc.teams[away_team_idx,]$w = wc.teams[away_team_idx,]$w +1
        } else {
            wc.teams[home_team_idx,]$d = wc.teams[home_team_idx,]$d +1
            wc.teams[away_team_idx,]$d = wc.teams[away_team_idx,]$d +1
        }
    }
    wc.teams <<- wc.teams %>% mutate(pts=3*w+d)
    wc.pool <<- wc.pool
}

In [17]:
create_8th_round <- function() {
    wc.qualified <- wc.teams %>% arrange(pool,desc(pts),desc(w),l) %>% group_by(pool) %>% 
                             mutate(pos=row_number()) %>% slice(1:2) %>% mutate(rank=paste0(pool,pos)) %>% ungroup()
    home_team <- c('A1','C1','E1','G1','B1','D1','F1','H1')
    away_team <- c('B2','D2','F2','H2','A2','C2','E2','G2')
    wc.8th <- data.frame(home_pos=home_team,away_pos=away_team,stringsAsFactors=F)
    wc.8th <<- wc.8th %>% inner_join(wc.qualified %>% select(team,rank),by=c('home_pos'='rank')) %>% rename(home_team=team) %>%
                     inner_join(wc.qualified %>% select(team,rank),by=c('away_pos'='rank')) %>% rename(away_team=team)
}

In [18]:
play_8th_round <- function() {
    for (row in 1:(dim(wc.8th)[1])) {
        home_team_idx = as.numeric(rownames(wc.teams[wc.teams$team==wc.8th[row,]$home_team,]))
        away_team_idx = as.numeric(rownames(wc.teams[wc.teams$team==wc.8th[row,]$away_team,]))

        wc.8th[row,'home_elo'] <- wc.teams[home_team_idx,]$elo
        wc.8th[row,'away_elo'] <- wc.teams[away_team_idx,]$elo
        wc.8th[row,'home_p'] <- 1/(1+10**((wc.8th[row,]$away_elo-wc.8th[row,]$home_elo)/400))
        wc.8th[row,'rand'] <- runif(1)
        wc.8th[row,'home_pts'] <- if_else(wc.8th[row,]$rand<=wc.8th[row,]$home_p,1,0)

        wc.teams[home_team_idx,'r8th'] <- wc.8th[row,]$home_pts
        wc.teams[away_team_idx,'r8th'] <- 1-wc.8th[row,]$home_pts

        nelo <- newelo(wc.8th[row,]$home_elo,wc.8th[row,]$away_elo,wc.8th[row,]$home_pts)
        wc.teams[home_team_idx,]$elo <- nelo[1]
        wc.teams[away_team_idx,]$elo <- nelo[2]
    }
    wc.teams <<- wc.teams
    wc.8th <<- wc.8th
}

In [19]:
create_4th_round <- function() {
    home_team <- c(if_else(wc.8th[1,]$home_pts==1,as.character(wc.8th[1,]$home_team),as.character(wc.8th[1,]$away_team)),
                   if_else(wc.8th[3,]$home_pts==1,as.character(wc.8th[3,]$home_team),as.character(wc.8th[3,]$away_team)),
                   if_else(wc.8th[5,]$home_pts==1,as.character(wc.8th[5,]$home_team),as.character(wc.8th[5,]$away_team)),
                   if_else(wc.8th[7,]$home_pts==1,as.character(wc.8th[7,]$home_team),as.character(wc.8th[7,]$away_team)) )

    away_team <- c(if_else(wc.8th[2,]$home_pts==1,as.character(wc.8th[2,]$home_team),as.character(wc.8th[2,]$away_team)),
                   if_else(wc.8th[4,]$home_pts==1,as.character(wc.8th[4,]$home_team),as.character(wc.8th[4,]$away_team)),
                   if_else(wc.8th[6,]$home_pts==1,as.character(wc.8th[6,]$home_team),as.character(wc.8th[6,]$away_team)),
                   if_else(wc.8th[8,]$home_pts==1,as.character(wc.8th[8,]$home_team),as.character(wc.8th[8,]$away_team)) )

    wc.4th <<- data.frame(home_team=home_team,away_team=away_team, stringsAsFactors=F) 
}

In [20]:
play_4th_round <- function() {
    for (row in 1:(dim(wc.4th)[1])) {
        home_team_idx = as.numeric(rownames(wc.teams[wc.teams$team==wc.4th[row,]$home_team,]))
        away_team_idx = as.numeric(rownames(wc.teams[wc.teams$team==wc.4th[row,]$away_team,]))

        wc.4th[row,'home_elo'] <- wc.teams[home_team_idx,]$elo
        wc.4th[row,'away_elo'] <- wc.teams[away_team_idx,]$elo
        wc.4th[row,'home_p'] <- 1/(1+10**((wc.4th[row,]$away_elo-wc.4th[row,]$home_elo)/400))
        wc.4th[row,'rand'] <- runif(1)
        wc.4th[row,'home_pts'] <- if_else(wc.4th[row,]$rand<=wc.4th[row,]$home_p,1,0)

        wc.teams[home_team_idx,'r4th'] <- wc.4th[row,]$home_pts
        wc.teams[away_team_idx,'r4th'] <- 1-wc.4th[row,]$home_pts

        nelo <- newelo(wc.4th[row,]$home_elo,wc.4th[row,]$away_elo,wc.4th[row,]$home_pts)
        wc.teams[home_team_idx,]$elo <- nelo[1]
        wc.teams[away_team_idx,]$elo <- nelo[2]
    }
    wc.teams <<- wc.teams
    wc.4th <<- wc.4th    
}

In [21]:
create_2th_round <- function() {
    home_team <- c(if_else(wc.4th[1,]$home_pts==1,as.character(wc.4th[1,]$home_team),as.character(wc.4th[1,]$away_team)),
                   if_else(wc.4th[3,]$home_pts==1,as.character(wc.4th[3,]$home_team),as.character(wc.4th[3,]$away_team)) )

    away_team <- c(if_else(wc.4th[2,]$home_pts==1,as.character(wc.4th[2,]$home_team),as.character(wc.4th[2,]$away_team)),
                   if_else(wc.4th[4,]$home_pts==1,as.character(wc.4th[4,]$home_team),as.character(wc.4th[4,]$away_team)) )

    wc.2th <<- data.frame(home_team=home_team,away_team=away_team, stringsAsFactors=F)   
}

In [22]:
play_2th_round <- function() {
    for (row in 1:(dim(wc.2th)[1])) {
        home_team_idx = as.numeric(rownames(wc.teams[wc.teams$team==wc.2th[row,]$home_team,]))
        away_team_idx = as.numeric(rownames(wc.teams[wc.teams$team==wc.2th[row,]$away_team,]))

        wc.2th[row,'home_elo'] <- wc.teams[home_team_idx,]$elo
        wc.2th[row,'away_elo'] <- wc.teams[away_team_idx,]$elo
        wc.2th[row,'home_p'] <- 1/(1+10**((wc.2th[row,]$away_elo-wc.2th[row,]$home_elo)/400))
        wc.2th[row,'rand'] <- runif(1)
        wc.2th[row,'home_pts'] <- if_else(wc.2th[row,]$rand<=wc.2th[row,]$home_p,1,0)

        wc.teams[home_team_idx,'r2th'] <- wc.2th[row,]$home_pts
        wc.teams[away_team_idx,'r2th'] <- 1-wc.2th[row,]$home_pts

        nelo <- newelo(wc.2th[row,]$home_elo,wc.2th[row,]$away_elo,wc.2th[row,]$home_pts)
        wc.teams[home_team_idx,]$elo <- nelo[1]
        wc.teams[away_team_idx,]$elo <- nelo[2]
    }
    wc.teams <<- wc.teams
    wc.2th <<- wc.2th
}

In [23]:
create_playoff_round <- function() {
    home_team <- c(if_else(wc.2th[1,]$home_pts==1,as.character(wc.2th[1,]$away_team),as.character(wc.2th[1,]$home_team)))
    away_team <- c(if_else(wc.2th[2,]$home_pts==1,as.character(wc.2th[2,]$away_team),as.character(wc.2th[2,]$home_team)))

    wc.playoff <<- data.frame(home_team=home_team,away_team=away_team, stringsAsFactors=F)    
}

In [24]:
play_playoff_round <- function() {
    home_team_idx = as.numeric(rownames(wc.teams[wc.teams$team==wc.playoff[1,]$home_team,]))
    away_team_idx = as.numeric(rownames(wc.teams[wc.teams$team==wc.playoff[1,]$away_team,]))

    wc.playoff[1,'home_elo'] <- wc.teams[home_team_idx,]$elo
    wc.playoff[1,'away_elo'] <- wc.teams[away_team_idx,]$elo
    wc.playoff[1,'home_p'] <- 1/(1+10**((wc.playoff[1,]$away_elo-wc.playoff[1,]$home_elo)/400))
    wc.playoff[1,'rand'] <- runif(1)
    wc.playoff[1,'home_pts'] <- if_else(wc.playoff[1,]$rand<=wc.playoff[1,]$home_p,1,0)

    wc.teams[home_team_idx,'rplayoff'] <- wc.playoff[1,]$home_pts
    wc.teams[away_team_idx,'rplayoff'] <- 1-wc.playoff[1,]$home_pts

    nelo <- newelo(wc.playoff[1,]$home_elo,wc.playoff[1,]$away_elo,wc.playoff[1,]$home_pts)
    wc.teams[home_team_idx,]$elo <- nelo[1]
    wc.teams[away_team_idx,]$elo <- nelo[2]

    wc.teams <<- wc.teams
    wc.playoff <<- wc.playoff
}

In [25]:
create_final_round <- function() {
    home_team <- c(if_else(wc.2th[1,]$home_pts==1,as.character(wc.2th[1,]$home_team),as.character(wc.2th[1,]$away_team)))
    away_team <- c(if_else(wc.2th[2,]$home_pts==1,as.character(wc.2th[2,]$home_team),as.character(wc.2th[2,]$away_team)))

    wc.final <<- data.frame(home_team=home_team,away_team=away_team, stringsAsFactors=F)   
}

In [26]:
play_final_round <- function() {
    home_team_idx = as.numeric(rownames(wc.teams[wc.teams$team==wc.final[1,]$home_team,]))
    away_team_idx = as.numeric(rownames(wc.teams[wc.teams$team==wc.final[1,]$away_team,]))

    wc.final[1,'home_elo'] <- wc.teams[home_team_idx,]$elo
    wc.final[1,'away_elo'] <- wc.teams[away_team_idx,]$elo
    wc.final[1,'home_p'] <- 1/(1+10**((wc.final[1,]$away_elo-wc.final[1,]$home_elo)/400))
    wc.final[1,'rand'] <- runif(1)
    wc.final[1,'home_pts'] <- if_else(wc.final[1,]$rand<=wc.final[1,]$home_p,1,0)

    wc.teams[home_team_idx,'rfinal'] <- wc.final[1,]$home_pts
    wc.teams[away_team_idx,'rfinal'] <- 1-wc.final[1,]$home_pts

    nelo <- newelo(wc.final[1,]$home_elo,wc.final[1,]$away_elo,wc.final[1,]$home_pts)
    wc.teams[home_team_idx,]$elo <- nelo[1]
    wc.teams[away_team_idx,]$elo <- nelo[2]
    
    wc.teams <<- wc.teams
    wc.final <<- wc.final
}

In [27]:
library(foreach)
library(doParallel)

Loading required package: iterators
Loading required package: parallel


In [28]:
no_cores <- detectCores() - 1  

In [32]:
tic('Outer')
cl <- makeCluster(no_cores, type="FORK")
registerDoParallel(cl)

for(r in 1:100) {
    tic(paste0('Inner',r))
    wc.teams.stats <- foreach(i=1:10000, .combine=rbind) %dopar%
    {
        set.seed(NULL)

        create_pool_round()
        play_pool_round()

        create_8th_round()
        play_8th_round()

        create_4th_round()
        play_4th_round()

        create_2th_round()
        play_2th_round()

        create_playoff_round()
        play_playoff_round()

        create_final_round()
        play_final_round()

        wc.teams.stats <- wc.teams %>% mutate(run=100*r+i)
    }
    toc()
    saveRDS(wc.teams.stats,paste0('wc.teams.stats.all.100k.',r))
}
stopCluster(cl)
toc()

Inner1: 186.596 sec elapsed
Inner2: 193.327 sec elapsed
Inner3: 189.41 sec elapsed
Inner4: 189.187 sec elapsed
Inner5: 187.525 sec elapsed
Inner6: 189.634 sec elapsed
Inner7: 188.596 sec elapsed
Inner8: 190.415 sec elapsed
Inner9: 189.882 sec elapsed
Inner10: 189.207 sec elapsed
Inner11: 187.224 sec elapsed
Inner12: 186.697 sec elapsed
Inner13: 189.851 sec elapsed
Inner14: 188.191 sec elapsed
Inner15: 186.811 sec elapsed
Inner16: 187.125 sec elapsed
Inner17: 187.242 sec elapsed
Inner18: 186.672 sec elapsed
Inner19: 187.347 sec elapsed
Inner20: 187.278 sec elapsed
Inner21: 187.223 sec elapsed
Inner22: 187.051 sec elapsed
Inner23: 185.574 sec elapsed
Inner24: 187.404 sec elapsed
Inner25: 185.412 sec elapsed
Inner26: 186.205 sec elapsed
Inner27: 187.402 sec elapsed
Inner28: 184.075 sec elapsed
Inner29: 187.307 sec elapsed
Inner30: 184.537 sec elapsed
Inner31: 186.903 sec elapsed
Inner32: 184.9 sec elapsed
Inner33: 185.291 sec elapsed
Inner34: 185.194 sec elapsed
Inner35: 185.011 sec elaps

In [56]:
wc.teams.stats <- readRDS('wc.teams.stats.all.100k.1')
for(r in 2:100) {
    filename <- paste0('wc.teams.stats.all.100k.',r)
    wc.teams.stats.small <- readRDS(filename) %>% mutate(run=10000*r+run)
    wc.teams.stats <- rbind(wc.teams.stats,wc.teams.stats.small)
}
dim(wc.teams.stats)

[1] 32000000       14

In [57]:
tail(wc.teams.stats)

,team,elo,match,w,l,d,pool,pts,r8th,r4th,r2th,rplayoff,rfinal,run
31999995,Nigeria,1185,3,1,2,0,D,3,NA,NA,NA,NA,NA,1010000
31999996,Tunisia,1196,3,1,2,0,G,3,NA,NA,NA,NA,NA,1010000
31999997,Morocco,1239,3,0,2,1,B,1,NA,NA,NA,NA,NA,1010000
31999998,Senegal,1211,3,0,3,0,H,0,NA,NA,NA,NA,NA,1010000
31999999,Saudi Arabia,1103,3,0,3,0,A,0,NA,NA,NA,NA,NA,1010000
32000000,Serbia,1232,3,0,1,2,E,2,NA,NA,NA,NA,NA,1010000


In [59]:
wc.teams.stats %>% select(team,r8th,r4th,r2th,rplayoff,rfinal) %>% 
                   replace_na(list(r8th=0,r4th=0,r2th=0,rplayoff=0,rfinal=0)) %>% 
                   group_by(team) %>% summarise_all(funs(mean(.,na.rm=T))) %>%
                   arrange(desc(rfinal),desc(rplayoff),desc(r2th),desc(r4th),desc(r8th))

team,r8th,r4th,r2th,rplayoff,rfinal
Brazil,0.711745,0.510491,0.353497,0.109708,0.237059
Spain,0.652667,0.400330,0.231379,0.102272,0.128018
France,0.541167,0.327238,0.167651,0.091965,0.089327
Germany,0.475471,0.288496,0.157915,0.073423,0.079741
Argentina,0.517639,0.285474,0.150738,0.073755,0.074172
England,0.574024,0.273100,0.147567,0.067309,0.073406
Belgium,0.528169,0.255562,0.132485,0.064160,0.062930
Portugal,0.434906,0.225396,0.104849,0.061103,0.048041
Peru,0.293456,0.150191,0.068393,0.039975,0.030285
Colombia,0.334165,0.145456,0.067202,0.037009,0.028728


In [ ]:
wc.teams.stats %>% filter(rfinal==1) %>% group_by(team) %>% summarize(n1=n()) %>% arrange(desc(n1)) %>% head(5)
predict.first <- wc.teams.stats %>% filter(rfinal==1) %>% group_by(team) %>% summarize(n1=n()) %>% arrange(desc(n1)) %>% head(1)

team,n1
Brazil,237059
Spain,128018
France,89327
Germany,79741
Argentina,74172


In [ ]:
wc.teams.stats %>% filter(rfinal==0) %>% anti_join(predict.first,by='team') %>% group_by(team) %>% summarize(n2=n()) %>% arrange(desc(n2)) %>% head(5)
predict.second <- wc.teams.stats %>% filter(rfinal==0) %>% anti_join(predict.first,by='team') %>% 
        group_by(team) %>% summarize(n2=n()) %>% arrange(desc(n2)) %>% head(1)

In [ ]:
wc.teams.stats %>% filter(rplayoff==1) %>% anti_join(predict.first,by='team') %>% anti_join(predict.second,by='team') %>%
        group_by(team) %>% summarize(n3=n()) %>% arrange(desc(n3)) %>% head(5)
predict.third <- wc.teams.stats %>% filter(rplayoff==1) %>% anti_join(predict.first,by='team') %>% anti_join(predict.second,by='team') %>% 
        group_by(team) %>% summarize(n3=n()) %>% arrange(desc(n3)) %>% head(1)

In [ ]:
wc.teams.stats %>% filter(rplayoff==0) %>% anti_join(predict.first,by='team') %>% anti_join(predict.second,by='team') %>% anti_join(predict.third,by='team') %>%
        group_by(team) %>% summarize(n4=n()) %>% arrange(desc(n4)) %>% head(5)
predict.four <- wc.teams.stats %>% filter(rplayoff==0) %>% anti_join(predict.first,by='team') %>% anti_join(predict.second,by='team') %>% anti_join(predict.third,by='team') %>%
        group_by(team) %>% summarize(n4=n()) %>% arrange(desc(n4)) %>% head(1)

In [ ]:
predict.first
predict.second
predict.third
predict.four

In [ ]:
tmp01 <- wc.teams.stats %>% filter(rfinal>=0) %>% select(team,run,rfinal) %>% spread(rfinal,team) %>% rename(t1='0',t2='1') %>%
                group_by(t1,t2) %>% summarize(n=n()) %>% ungroup()
tmp02 <- tmp01 %>% inner_join(tmp01,by=c('t1'='t2','t2'='t1')) %>% mutate(total=n.x+n.y)
tmp02 %>% arrange(desc(total)) %>% head()

In [ ]:
tmp01 <- wc.teams.stats %>% filter(rplayoff>=0) %>% select(team,run,rplayoff) %>% spread(rplayoff,team) %>% rename(t1='0',t2='1') %>%
                group_by(t1,t2) %>% summarize(n=n()) %>% ungroup()
tmp02 <- tmp01 %>% inner_join(tmp01,by=c('t1'='t2','t2'='t1')) %>% mutate(total=n.x+n.y)
tmp02 %>% arrange(desc(total)) %>% head()

In [ ]:
tmp02 %>% filter(!t1 %in% c('Spain','Brazil')) %>% filter(!t2 %in% c('Spain','Brazil')) %>% arrange(desc(total)) %>% head()

In [ ]:
matches.selected %>% filter(home_team %in% c('France','Argentina')) %>% filter(away_team %in% c('France','Argentina')) %>%
            mutate(winner=if_else(home_score>away_score,home_team,if_else(home_score<away_score,away_team,NA_character_))) %>%
            group_by(winner) %>% summarize(count=n())